<a href="https://colab.research.google.com/github/yugendrasai-ui/ai-video-pipeline/blob/main/AI_Video_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
# Install required libraries
!pip install google-generativeai edge-tts moviepy requests nest_asyncio opencv-python-headless


In [80]:
from google.colab import userdata
import google.generativeai as genai

# Load Gemini API Key
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

if not GEMINI_API_KEY:
    raise ValueError("Gemini API Key not found")

# Configure Gemini
genai.configure(api_key=GEMINI_API_KEY)

# Load Pexels API Key
PEXELS_KEY = userdata.get("PEXELS_KEY")

if not PEXELS_KEY:
    raise ValueError("Pexels API Key not found")

print("✅ API Keys Loaded Successfully")


✅ API Keys Loaded Successfully


In [81]:
# Load Gemini Model
model = genai.GenerativeModel("models/gemini-2.5-flash")

print("✅ Gemini Model Ready")




✅ Gemini Model Ready


In [82]:
def generate_script(topic):

    prompt = f"""
    Write a natural YouTube voice script about "{topic}"

    Rules:
    - Simple English
    - No bullet points
    - No headings
    - No symbols
    - Friendly tone
    - About 1 minute
    """

    response = model.generate_content(prompt)

    return response.text.strip()


In [83]:
import re

def clean_script(text):

    # Remove special symbols
    text = re.sub(r"[*#>`_~]", "", text)

    # Replace extra punctuation
    text = re.sub(r"[!?:;]+", ".", text)

    # Remove extra spaces
    text = re.sub(r"\s+", " ", text)

    return text.strip()


In [84]:
import edge_tts
import asyncio
import nest_asyncio

nest_asyncio.apply()

async def text_to_voice(text):

    voice = edge_tts.Communicate(
        text,
        voice="en-IN-NeerjaNeural"   # Indian female voice
    )

    await voice.save("voice.mp3")

    print("✅ Voice Created")


In [86]:
import requests
from urllib.parse import quote

def get_images(topic, count=12):

    topic = quote(topic)

    url = f"https://api.pexels.com/v1/search?query={topic}&per_page={count}"

    headers = {
        "Authorization": PEXELS_KEY
    }

    res = requests.get(url, headers=headers)

    if res.status_code != 200:
        print("❌ Pexels Error")
        return []

    data = res.json()

    images = []

    for i, photo in enumerate(data["photos"]):

        img_url = photo["src"]["large"]

        img = requests.get(img_url).content

        name = f"img{i}.jpg"

        with open(name, "wb") as f:
            f.write(img)

        images.append(name)

    print("✅ Images Downloaded:", len(images))

    return images


In [87]:
from moviepy.editor import *
import cv2

def make_video(images, audio_file):

    # Load Audio
    audio = AudioFileClip(audio_file)

    duration = audio.duration

    print("Audio Duration:", duration)

    # Time per image
    img_time = duration / len(images)

    clips = []

    for img in images:

        frame = cv2.imread(img)

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        frame = cv2.resize(frame, (1280,720))

        clip = ImageClip(frame).set_duration(img_time)

        clips.append(clip)

    # Combine all images
    video = concatenate_videoclips(clips, method="compose")

    # Add audio
    final = video.set_audio(audio)

    # Export
    final.write_videofile(
        "final_video.mp4",
        fps=24,
        codec="libx264",
        audio_codec="aac"
    )

    return "final_video.mp4"


In [88]:
def generate_seo(script):

    prompt = f"""
    From this script generate:

    1. YouTube Title (max 70 chars)
    2. Description (3 lines)
    3. 10 Tags (comma separated)

    Script:
    {script}
    """

    res = model.generate_content(prompt)

    return res.text.strip()


In [89]:
topic = "Artificial Intelligence in 2026"

print("🔹 Generating Script...")
raw_script = generate_script(topic)

print("🔹 Cleaning Script...")
script = clean_script(raw_script)

print("🔹 Creating Voice...")
await text_to_voice(script)

print("🔹 Downloading Images...")
images = get_images(topic)

print("🔹 Creating Video...")
video = make_video(images, "voice.mp3")

print("🔹 Generating SEO...")
seo = generate_seo(script)

print("\n📌 SEO CONTENT:\n")
print(seo)

print("\n✅ DONE! Video Created:", video)


🔹 Generating Script...
🔹 Cleaning Script...
🔹 Creating Voice...
✅ Voice Created
🔹 Downloading Images...
✅ Images Downloaded: 12
🔹 Creating Video...
Audio Duration: 104.52000000000001
Moviepy - Building video final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video final_video.mp4



Moviepy - Done !
Moviepy - video ready final_video.mp4
🔹 Generating SEO...

📌 SEO CONTENT:

Here's the generated content based on your script:

1.  **YouTube Title**
    AI in 2026: Your Smartest, Most Personal Helper!

2.  **Description**
    Ever wonder what Artificial Intelligence will be like in just a couple of years? We're fast-forwarding to 2026 to explore how AI will seamlessly integrate into our daily lives, making things simpler and more personal.
    From super-smart personal assistants that know you better than ever to AI tools that free you up for creative work and custom learning experiences, it's all about making life easier and more efficient.
    What are your thoughts on AI in 2026? Let us know in the comments below! Don't forget to like and subscribe for more.

3.  **10 Tags**
    AI 2026, Artificial Intelligence, Future of AI, AI personal assistant, AI at work, AI in learning, Future technology, AI explained, Smart technology, Tech trends

✅ DONE! Video Created: fin

In [90]:
from IPython.display import Video, display

display(Video("final_video.mp4", embed=True))
